# 🚀 StairVision Enhanced Training - 300 Epochs

## Improvements Over Previous Version:
- **300 epochs** (3x more training)
- **YOLOv8s** (small model - more accurate)
- **Enhanced augmentation** (stronger generalization)
- **Extended patience** (50 epochs)
- **Better validation**

**Expected Results:**
- mAP@50: 98-99% (vs 97% current)
- Fewer false positives
- Better detection at various angles
- Training Time: 3-4 hours on Colab T4 GPU

## 📦 Cell 1: Mount Google Drive & Install Dependencies

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Ultralytics (YOLOv8)
%pip install ultralytics --quiet

# Import libraries
from ultralytics import YOLO
import os
import shutil
from pathlib import Path

print("✅ Setup complete!")

## 📁 Cell 2: Dataset Preparation

Upload your stair dataset to Google Drive, then update the path below.

In [ ]:
# IMPORTANT: Update this path to your dataset location in Google Drive
DATASET_PATH = '/content/drive/MyDrive/RGB-D stair dataset'

# Copy dataset to Colab local storage for faster access
!mkdir -p /content/stairs_dataset
!cp -r "{DATASET_PATH}/train" /content/stairs_dataset/
!cp -r "{DATASET_PATH}/val" /content/stairs_dataset/
!cp -r "{DATASET_PATH}/test" /content/stairs_dataset/

print("✅ Dataset copied to local storage")

# Verify dataset structure
!ls -lh /content/stairs_dataset/

## 📋 Cell 3: Create data.yaml Configuration

In [ ]:
# Create YAML configuration file
yaml_content = """
# StairVision Dataset Configuration
path: /content/stairs_dataset
train: train/images
val: val/images
test: test/images

# Classes
names:
  0: stairs
"""

with open('/content/stairs_dataset/data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created")
!cat /content/stairs_dataset/data.yaml

## 🎯 Cell 4: Enhanced Training - 300 Epochs

In [ ]:
# Initialize YOLOv8s (small model - more accurate than nano)
model = YOLO('yolov8s.pt')

# ENHANCED TRAINING - 300 EPOCHS
results = model.train(
    data='/content/stairs_dataset/data.yaml',
    
    # Training duration
    epochs=300,              # 3x more training!
    patience=50,             # Wait 50 epochs before early stopping
    
    # Model configuration
    imgsz=640,              # Image size
    batch=16,               # Batch size (adjust if OOM)
    device=0,               # GPU
    
    # Output configuration
    project='stair_detection_v2',
    name='yolov8s_300epochs',
    save=True,
    plots=True,
    
    # Single class detection
    single_cls=True,
    exist_ok=True,
    
    # ENHANCED DATA AUGMENTATION
    hsv_h=0.03,             # Hue augmentation (increased)
    hsv_s=0.9,              # Saturation augmentation (increased)
    hsv_v=0.6,              # Value augmentation (increased)
    degrees=20,             # Rotation degrees (increased)
    translate=0.2,          # Translation
    scale=0.9,              # Scaling (increased range)
    shear=5.0,              # Shear transformation
    perspective=0.001,      # Perspective transformation
    flipud=0.0,             # Vertical flip (stairs don't flip)
    fliplr=0.5,             # Horizontal flip
    mosaic=1.0,             # Mosaic augmentation
    mixup=0.3,              # Mixup augmentation (increased)
    copy_paste=0.2,         # Copy-paste augmentation
    
    # Optimization
    optimizer='AdamW',      # Better optimizer
    lr0=0.001,              # Initial learning rate
    lrf=0.01,               # Final learning rate
    momentum=0.95,          # Momentum
    weight_decay=0.0005,    # Weight decay
    warmup_epochs=5,        # Warmup epochs
    warmup_momentum=0.8,    # Warmup momentum
    
    # Loss weights
    box=7.5,                # Box loss weight
    cls=0.5,                # Classification loss weight
    dfl=1.5,                # Distribution focal loss weight
    
    # Validation
    val=True,               # Validate during training
    save_period=25,         # Save checkpoint every 25 epochs
    
    # Performance
    cache=True,             # Cache images for faster training
    workers=8,              # Number of workers
    verbose=True,           # Verbose output
)

print("\n🎉 Training Complete!")
print(f"Results saved to: {results.save_dir}")

## 📊 Cell 5: View Training Results

In [ ]:
from IPython.display import Image, display

# Display training curves
print("📈 Training Curves:")
display(Image(filename=f'{results.save_dir}/results.png'))

print("\n📉 Confusion Matrix:")
display(Image(filename=f'{results.save_dir}/confusion_matrix.png'))

print("\n🎯 Validation Batch Predictions:")
display(Image(filename=f'{results.save_dir}/val_batch0_pred.jpg'))

## ✅ Cell 6: Validate Model Performance

In [ ]:
# Validate on test set
metrics = model.val(data='/content/stairs_dataset/data.yaml', split='test')

print("\n📊 Final Model Performance:")
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

if metrics.box.map50 > 0.98:
    print("\n🎉 EXCELLENT! Model achieved >98% mAP@50")
elif metrics.box.map50 > 0.95:
    print("\n✅ GOOD! Model achieved >95% mAP@50")
else:
    print("\n⚠️ Model may need more training or data")

## 📱 Cell 7: Export to TensorFlowLite (Float32)

In [ ]:
# Export best model to TFLite format
best_model = YOLO(f'{results.save_dir}/weights/best.pt')

# Export to TFLite (Float32 for Android)
tflite_path = best_model.export(
    format='tflite',
    imgsz=640,
    int8=False,  # Float32 model (more accurate)
)

print(f"\n✅ TFLite model exported to: {tflite_path}")

# Copy to Google Drive
output_name = 'stair_yolo_v2_300epochs_float32.tflite'
drive_path = f'/content/drive/MyDrive/{output_name}'
!cp "{tflite_path}" "{drive_path}"

print(f"\n📥 Model saved to Google Drive: {output_name}")
print("\nNext steps:")
print("1. Download this file from Google Drive")
print("2. Place in: StairVision/app/src/main/assets/")
print("3. Update YoloDetector.kt MODEL_NAME if needed")
print("4. Build and test!")

## 🧪 Cell 8: Test on Sample Images

In [ ]:
# Test on validation images
import glob
from pathlib import Path

# Get some test images
test_images = glob.glob('/content/stairs_dataset/val/images/*.jpg')[:5]

print(f"Testing on {len(test_images)} sample images...\n")

for img_path in test_images:
    results = best_model.predict(
        source=img_path,
        conf=0.85,  # Use your threshold
        save=True,
        project='test_predictions',
        name='samples'
    )
    
    # Print detection info
    img_name = Path(img_path).name
    if len(results[0].boxes) > 0:
        max_conf = max([box.conf.item() for box in results[0].boxes])
        print(f"✅ {img_name}: Detected stairs (conf: {max_conf:.3f})")
    else:
        print(f"⚪ {img_name}: No stairs detected")

print("\n📁 Predictions saved to: test_predictions/samples/")

## 💾 Cell 9: Save All Files to Google Drive

In [ ]:
# Create backup folder in Google Drive
backup_folder = '/content/drive/MyDrive/StairVision_v2_300epochs_backup'
!mkdir -p "{backup_folder}"

# Copy all training results
!cp -r "{results.save_dir}" "{backup_folder}/training_results"

# Copy test predictions
!cp -r test_predictions "{backup_folder}/test_predictions"

print("✅ All training results backed up to Google Drive")
print(f"Location: {backup_folder}")
print("\n📦 Contents:")
!ls -lh "{backup_folder}"

## 📈 Cell 10: Performance Comparison

In [ ]:
print("📊 Expected Improvements:")
print("\nPrevious Model (100 epochs, YOLOv8n):")
print("  - mAP@50: ~97%")
print("  - False positives: Moderate")
print("  - Model size: 6MB")
print("\nNew Model (300 epochs, YOLOv8s):")
print("  - mAP@50: ~98-99%")
print("  - False positives: Minimal")
print("  - Model size: ~22MB")
print("  - Better generalization")
print("  - More robust to lighting/angles")
print("\n✅ Recommended threshold: 0.75-0.85")